In [2]:
%pylab inline
#Autoreload so changing py files works
%load_ext autoreload
%autoreload 2
from Util import *
from superpoint import *
from VideoStabilization import *

Populating the interactive namespace from numpy and matplotlib


C:\Users\MasterMatthew\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['resize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:


class SUPERFeatureExtractor(BaseFeatureExtractor):
    def __init__(self):
        self.superpt = SuperPointFrontend(weights_path='./superpoint_v1.pth',
                      nms_dist=4,
                      conf_thresh=0.05,
                      nn_thresh=0.7,
                      cuda=False)
    
    def extractFeatures(self, image):
        return self.superpt.run(image)
def match(desc1, desc2):
    pTracker = PointTracker(2,0.7)
    return pTracker.nn_match_two_way(desc1, desc2, 0.4)
def convertToPoints(pts1, pts2, matches):
    pointsx1 = pts1[0]
    pointsy1 = pts1[1]
    pointsx2 = pts2[0]
    pointsy2 = pts2[1]
    pts1_pair = []
    pts2_pair = []
    #print(len(pointsx1))
    #print(len(pointsx2))
    #print(np.max(matches[0]))
    #print(np.max(matches[1]))
    for i in range(matches.shape[1]):
        queryIdx=int(matches[1][i])
        trainIdx=int(matches[0][i])

        size_1 = len(pointsx1)-1
        size_2 = len(pointsx2)-1

        if queryIdx>size_1 or trainIdx>size_2:
            continue

        pt1 = (int(pointsx1[queryIdx]), int(pointsy1[queryIdx]))
        pt2 = (int(pointsx2[trainIdx]), int(pointsy2[trainIdx]))
        pts1_pair.append(pt1)
        pts2_pair.append(pt2)
    return np.array(pts1_pair), np.array(pts2_pair)

In [4]:
inputpath = 0
resize = (800, 600)
skip = 0
max_frames = 9999
show = True

SUPERPOINT=False
if SUPERPOINT:
    extractor = SUPERFeatureExtractor()
else:
    extractor = SIFTFeatureExtractor()
    matcher = BFMatcherHomographyGenerator()

def read():
    ret, frame = cap.read()
    if frame.shape[0] < frame.shape[1]:
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 1)
    if resize:
        frame = cv2.resize(frame, resize)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color = frame#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return color, gray

cap = cv2.VideoCapture(inputpath)
correctedFrames = []
count = 0
H = np.array([[1,0,0],[0,1,0],[0,0,1]])

#Read first frame
color1, gray1 = read()
correctedFrames.append(color1)
kps1, desc1 = extractor.extractFeatures(gray1)

while cap.isOpened():
    #Read in next frame
    color2, gray2 = read()
    count = count + 1
    #Allow frame skipping
    if count > max_frames: break
    if count % skip == 0:
        kps2, desc2 = extractor.extractFeatures(gray2)
        if SUPERPOINT:
            matches = match(desc1, desc2)
            pair1, pair2 = convertToPoints(kps1, kps2, matches)
            HG, mask = cv2.findHomography(pair2, pair1, cv2.RANSAC, 5.0)
        else:
            HG = matcher.findHomography(kps2, desc2, kps1, desc1)
        H = H @ HG
        corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
        correctedFrames.append(corrected)
        kps1 = kps2
        desc1 = desc2
        
        if show:
            cv2.imshow('frame', corrected)
    #Press space to end
    if cv2.waitKey(32) == 32:
        break
    
cap.release()